In [ ]:
import matplotlib.pyplot as plt
import cv2

image = cv2.imread('/Users/shingkai/code/personal-projects/mk8/race_videos/training/mirror_flower_cup/p1_first_item/0/001825.png')
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
from enum import Enum

class Item(int, Enum):
    BANANA = 1
    TRIPLE_BANANA = 2
    GREEN_SHELL = 3
    TRIPLE_GREEN_SHELL = 4
    RED_SHELL = 5
    TRIPLE_RED_SHELL = 6
    BLUE_SHELL = 7
    BOB_OMB = 8
    MUSHROOM = 9
    TRIPLE_MUSHROOM = 10
    GOLDEN_MUSHROOM = 11
    BULLET_BILL = 12
    BLOOPER = 13
    LIGHTNING = 14
    STAR = 15
    FIRE_FLOWER = 16
    BOOMERANG = 17
    PIRANHA_PLANT = 18
    SUPER_HORN = 19
    CRAZY_EIGHT = 20
    COIN = 21
    FEATHER = 22
    BOO = 23
    NONE = 24

In [ ]:
GREEN_SHELL : {[(3510,4002)]}
RED_SHELL : {[(1825,1955)]}
COIN : {[(2693,2860),(3443,3483)]}


In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

def load_images(folder_path):
    images = []
    file_paths = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(folder_path, filename)
            try:
                img = Image.open(file_path).convert('RGB')
                img = img.resize((100, 100))  # Resize for consistency
                images.append(img)
                file_paths.append(file_path)
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
    return images, file_paths

def extract_features(images):
    features = []
    for img in images:
        hist = img.histogram()
        features.append(hist)
    return np.array(features)

def cluster_images(folder_path, n_clusters):
    # Load images
    print("Loading images...")
    images, file_paths = load_images(folder_path)
    
    # Extract features
    print("Extracting features...")
    features = extract_features(images)
    
    # Normalize features
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features)
    
    # Perform K-means clustering
    print("Clustering images...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(normalized_features)
    
    # Organize images into clusters
    clusters = {i: [] for i in range(n_clusters)}
    for file_path, label in zip(file_paths, cluster_labels):
        clusters[label].append(file_path)
    
    # Print results
    for cluster, files in clusters.items():
        print(f"Cluster {cluster}: {len(files)} images")
        # Optionally, you can print file names or move files to cluster-specific folders here
    
    return clusters
    


In [ ]:
folder_path = "/Users/shingkai/code/personal-projects/mk8/race_videos/training/mirror_flower_cup/p1_first_item/0/"
n_clusters = 50
clustered_images = cluster_images(folder_path, n_clusters)

In [ ]:
for image in clustered_images[2][:5]:
    plt.figure()
    plt.imshow(cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
plt.subplots(figsize=(16, 64))
plt.axis('off')

for i in range(50):
    for j in range(10):
        plt.subplot(50,10,i*10+j+1)
        plt.axis('off')
        plt.imshow(cv2.cvtColor(cv2.imread(clustered_images[i][j]), cv2.COLOR_BGR2RGB))

plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import time
import copy

# # Check if GPU is available (either cuda for nvidia or mps for apple silicon)
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else  torch.device("cpu"))


# Define transformations for the dataset (same for train and validation initially)
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Set the directory path where the dataset is stored (single directory with subdirectories for each class)
data_dir = '/Users/shingkai/code/personal-projects/mk8/training_data/items'
# data_dir = '/home/itsgrimetime/code/MarioKart8CV/items'

# Load the dataset from the directory
full_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

# Split the dataset into train and validation sets (e.g., 80% train, 20% validation)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create dataloaders for train and validation sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Dictionary to hold the dataloaders
dataloaders = {
    'train': train_loader,
    'val': val_loader
}

# Sizes of the datasets
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset)
}

# Class names (same as subdirectory names)
class_names = full_dataset.classes
print(f"class names: {class_names}")

# Load the pretrained VGG16 model
model = models.vgg16(pretrained=True)

# Freeze all layers in the network except the final classifier layer
for param in model.parameters():
    param.requires_grad = False

# Modify the classifier to fit our dataset (assuming the number of classes is determined by the dataset)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(class_names))

# Move the model to the appropriate device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)

# Learning rate scheduler to decrease learning rate by a factor of 0.1 every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Train the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Track loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Adjust learning rate
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.float() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it has improved
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load the best model weights
    model.load_state_dict(best_model_wts)
    return model



In [ ]:
# Train the model
num_epochs = 25
best_model = train_model(model, criterion, optimizer, scheduler, num_epochs)

# Save the trained model
torch.save(best_model.state_dict(), 'best_model_vgg16.pth')

In [ ]:
# class_names = ['1','3','5','9','16','21','None']

class_names = ['1', '16', '21', '3', '5', '9', 'None']

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np

# # Check if GPU is available (either cuda for nvidia or mps for apple silicon)
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else  torch.device("cpu"))


# Load the model architecture
model = models.vgg16(pretrained=False)
num_features = model.classifier[6].in_features

# Modify the classifier to fit the number of classes in your model
model.classifier[6] = torch.nn.Linear(num_features, len(class_names))  # class_names should have the number of classes in your dataset

# Load the saved weights

model.load_state_dict(torch.load('/Users/shingkai/code/personal-projects/mk8/models/item_classifier_vgg16.pth', map_location=device))
# model.load_state_dict(torch.load('/home/itsgrimetime/code/MarioKart8CV/notebooks/best_model_vgg16.pth', map_location=device))



# Set the model to evaluation mode
model.eval()

# Move the model to the appropriate device (MPS or CPU)
model = model.to(device)

# Define the transformation to be applied to the input image (same as used during training)
preprocess = transforms.Compose([
    transforms.ToPILImage(),  # Convert numpy array (cv2) to PIL image
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def predict_image(frame):
    
    # Preprocess the image
    img = preprocess(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Add batch dimension (PyTorch models expect a batch of images)
    img = img.unsqueeze(0)
    
    # Move the image to the same device as the model (MPS or CPU)
    img = img.to(device)
    
    # Turn off gradients for inference (faster and reduces memory usage)
    with torch.no_grad():
        outputs = model(img)
    
    # Get the predicted class (with the highest score)
    _, predicted_class = torch.max(outputs, 1)
    
    # Map the predicted class index to the class name
    predicted_class_name = class_names[predicted_class.item()]
    
    return predicted_class_name



In [ ]:
import matplotlib.pyplot as plt
import cv2

# Example of running inference
image_path = '/Users/shingkai/code/personal-projects/mk8/race_videos/training/mirror_flower_cup/p1_first_item/0/020094.png'
# image_path = '/home/itsgrimetime/code/MarioKart8CV/race_videoss/training/item1/Player.P1/000922.png'

frame = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
predicted_label = predict_image(frame)
print(f'The predicted label for the image is: {predicted_label}')

In [ ]:
image_path = '/Users/shingkai/code/personal-projects/mk8/race_videos/training/mirror_flower_cup/p1_first_item/0/026030.png'
frame = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
predicted_label = predict_image(frame)
print(f'The predicted label for the image is: {predicted_label}')

In [ ]:
image_path = '/Users/shingkai/code/personal-projects/mk8/race_videos/training/mirror_flower_cup/p1_first_item/0/025855.png'
frame = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
predicted_label = predict_image(frame)
print(f'The predicted label for the image is: {predicted_label}')

In [ ]:
image_path = '/Users/shingkai/code/personal-projects/mk8/race_videos/training/mirror_flower_cup/p1_first_item/0/029638.png'
frame = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
predicted_label = predict_image(frame)
print(f'The predicted label for the image is: {predicted_label}')

# ResNeXt-50 Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import os

# Check if MPS (Apple Silicon) or CUDA is available
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set hyperparameters
num_classes = 7  # Change this to the number of classes in your dataset
batch_size = 32
num_epochs = 10
learning_rate = 0.001
val_split = 0.2  # 20% of the data will be used for validation

# Define the data transforms for training and validation
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset using ImageFolder
data_dir = '/Users/shingkai/code/personal-projects/mk8/training_data/items'  # Update this to your dataset path
dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

# Calculate the sizes for training and validation
num_val_samples = int(val_split * len(dataset))
num_train_samples = len(dataset) - num_val_samples

# Create training and validation datasets
train_dataset, val_dataset = random_split(dataset, [num_train_samples, num_val_samples])

# Create the data loaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
}

# Get the number of classes
class_names = dataset.classes
print(f"class names: {class_names}")
num_classes = len(class_names)

# Load the ResNeXt-50 model, pretrained on ImageNet
# model = models.resnext50_32x4d(pretrained=True)
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer to match the number of classes in your dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to the device (MPS, CUDA, or CPU)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Function to train the model
def train_model(model, criterion, optimizer, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Calculate the loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it has better accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [ ]:
# Train the model
best_model = train_model(model, criterion, optimizer, num_epochs)

# Save the best model weights
torch.save(best_model.state_dict(), 'item_classifier_resnet18.pth')

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np

classes = ['01', '02', '03', '04', '05', '07', '09', '10', '11', '12', '13', '15', '16', '18', '19', '21', '23', '24']

# # Check if GPU is available (either cuda for nvidia or mps for apple silicon)
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else  torch.device("cpu"))


# Load the model architecture
# model = models.resnext50_32x4d(pretrained=False)
model = models.resnet18(pretrained=True)
 # Adjust final layer to match output classes
model.fc = torch.nn.Linear(model.fc.in_features, len(class_names)) 
# Load the saved weights
model.load_state_dict(torch.load('/Users/shingkai/code/personal-projects/mk8/models/item_classifier_resnet18.pth', map_location=device))
# model.load_state_dict(torch.load('/home/itsgrimetime/code/MarioKart8CV/notebooks/item_classifier_resnet18.pth', map_location=device))

# Set the model to evaluation mode
model.eval()

# Move the model to the appropriate device (MPS or CPU)
model = model.to(device)

# Define the transformation to be applied to the input image (same as used during training)
preprocess = transforms.Compose([
    transforms.ToPILImage(),  # Convert numpy array (cv2) to PIL image
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def predict_from_frame(frame):
    
    # Preprocess the image
    img = preprocess(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Add batch dimension (PyTorch models expect a batch of images)
    img = img.unsqueeze(0)
    
    # Move the image to the same device as the model (MPS or CPU)
    img = img.to(device)
    
    # Turn off gradients for inference (faster and reduces memory usage)
    with torch.no_grad():
        outputs = model(img)
    
    # Get the predicted class (with the highest score)
    _, predicted_class = torch.max(outputs, 1)
    
    # Map the predicted class index to the class name
    predicted_class_name = class_names[predicted_class.item()]
    
    return predicted_class_name


In [ ]:
import matplotlib.pyplot as plt
import cv2

classes = ['01', '02', '03', '04', '05', '07', '09', '10', '11', '12', '13', '15', '16', '18', '19', '21', '23', '24']

# Example of running inference
image_path = '/Users/shingkai/code/personal-projects/mk8/race_videos/training/mirror_flower_cup/p1_first_item/0/020094.png'
# image_path = '/home/itsgrimetime/code/MarioKart8CV/race_videoss/training/item1/Player.P1/000922.png'

frame = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
predicted_label = predict_from_frame(frame)

print(f'The predicted label for the image is: {predicted_label}')

# SqueezeNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import cv2
import numpy as np

# Set up parameters
img_width, img_height = 112, 112  # SqueezeNet default input size
batch_size = 32
epochs = 5
data_dir = '/Users/shingkai/code/personal-projects/mk8/training_data/items'
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else  torch.device("cpu"))

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = self._load_images()

    def _load_images(self):
        images = []
        for cls_name in self.classes:
            cls_dir = os.path.join(self.data_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                img_path = os.path.join(cls_dir, img_name)
                images.append((img_path, self.class_to_idx[cls_name]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Data transforms
data_transforms = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
dataset = CustomDataset(data_dir, transform=data_transforms)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load pre-trained SqueezeNet model
model = models.squeezenet1_1(pretrained=True)

# Modify the classifier
model.classifier[1] = nn.Conv2d(512, len(dataset.classes), kernel_size=(1,1), stride=(1,1))
model.num_classes = len(dataset.classes)

# Move model to device
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")

# Save the model
torch.save(model.state_dict(), 'squeezenet_pytorch.pth')

print("Training completed. Model saved as 'squeezenet_pytorch.pth'")


In [ ]:
classes = ['01', '02', '03', '04', '05', '07', '09', '10', '11', '12', '13', '15', '16', '18', '19', '21', '23', '24']

# Set up parameters
img_width, img_height = 112, 112  # SqueezeNet default input size
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else  torch.device("cpu"))

# Load the pre-trained SqueezeNet model architecture
model = models.squeezenet1_1(pretrained=False)

# Modify the classifier to match your number of classes
num_classes = len(classes)  # Replace with the number of classes in your dataset
model.classifier[1] = torch.nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
model.num_classes = num_classes

# Load the saved state dict
model.load_state_dict(torch.load('squeezenet_pytorch.pth', map_location=device))
model = model.to(device)
model.eval()

# Define the transformation for input images
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((img_width, img_height)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Function to predict on a single image
def predict_image(image_path, model, transform):
    image = cv2.imread(image_path)
    predict_frame(image, model, transform)


# Function to predict on a cv2 frame
def predict_frame(frame, model=model, preprocess=preprocess):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = preprocess(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
        confidence = probabilities[predicted_class].item()
    
    return predicted_class, confidence

In [ ]:
import matplotlib.pyplot as plt
import cv2

classes = ['01', '02', '03', '04', '05', '07', '09', '10', '11', '12', '13', '15', '16', '18', '19', '21', '23', '24']

# Example of running inference
image_path = '/Users/shingkai/code/personal-projects/mk8/race_videos/training/mirror_flower_cup/p1_first_item/0/020094.png'


# image_path = '/Users/shingkai/code/personal-projects/mk8/race_videos/training/mirror_flower_cup/p1_first_item/0/026030.png'
# image_path = '/Users/shingkai/code/personal-projects/mk8/race_videos/training/mirror_flower_cup/p1_first_item/0/025855.png'


# image_path = '/home/itsgrimetime/code/MarioKart8CV/race_videoss/training/item1/Player.P1/000922.png'

frame = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
prediction, confidence = predict_frame(frame)
predicted_label = classes[prediction]

print(f'The predicted label for the image is: {predicted_label} ({confidence:.2f})')


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import os

# Check if MPS (Apple Silicon) or CUDA is available
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set hyperparameters
img_width, img_height = 112, 112
batch_size = 32
num_epochs = 10
learning_rate = 0.001
val_split = 0.2  # 20% of the data will be used for validation

# Define the data transforms for training and validation
data_transforms = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset using ImageFolder
data_dir = '/Users/shingkai/code/personal-projects/mk8/training_data/items'  # Update this to your dataset path
dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

# Calculate the sizes for training and validation
num_val_samples = int(val_split * len(dataset))
num_train_samples = len(dataset) - num_val_samples

# Create training and validation datasets
train_dataset, val_dataset = random_split(dataset, [num_train_samples, num_val_samples])

# Create the data loaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
}

# Get the number of classes
class_names = dataset.classes
print(f"class names: {class_names}")
num_classes = len(class_names)

# Load the ResNeXt-50 model, pretrained on ImageNet
# model = models.resnext50_32x4d(pretrained=True)
# model = models.resnet18(pretrained=True)
model = models.squeezenet1_1(pretrained=True)


# Modify the final fully connected layer to match the number of classes in your dataset
# model.fc = nn.Linear(model.fc.in_features, num_classes)

# Modify the classifier
model.classifier[1] = nn.Conv2d(512, len(dataset.classes), kernel_size=(1,1), stride=(1,1))
model.num_classes = len(dataset.classes)

# Move the model to the device (MPS, CUDA, or CPU)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Function to train the model
def train_model(model, criterion, optimizer, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Calculate the loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it has better accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
# Train the model
best_model = train_model(model, criterion, optimizer, num_epochs)

# Save the best model weights
torch.save(best_model.state_dict(), 'item_classifier_squeezenet.pth')

# MobileNet v3

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import os

# Check if MPS (Apple Silicon) or CUDA is available
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Data transformations (96x96 resizing)
data_transforms = transforms.Compose([
    transforms.Resize((96, 96)),            # Resize to 96x96 pixels
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset (without splitting into train/val yet)
data_dir = '/Users/shingkai/code/personal-projects/mk8/training_data/positions/train'
# data_dir = '/home/itsgrimetime/code/MarioKart8CV/items'
full_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

# Split dataset into train and validation sets
train_size = int(0.8 * len(full_dataset))   # 80% training
val_size = len(full_dataset) - train_size   # 20% validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Data loaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
}

# Get number of classes
num_classes = len(full_dataset.classes)

# Load MobileNetV3-large pre-trained model
model = models.mobilenet_v3_large(pretrained=True)

# Modify the last fully connected layer to match the number of classes
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)

# Move model to the GPU if available
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, criterion, optimizer, dataloaders, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f'Best val Acc: {best_acc:4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Fine-tune the model
model = train_model(model, criterion, optimizer, dataloaders, num_epochs=5)

# Save the trained model
torch.save(model.state_dict(), '../models/position_classifier_mobilenetv3.pth')

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import cv2
from PIL import Image
import numpy as np

classes = ['00','01',"02","03","04","05","06","07","08","09","10","11","12"]

# Check if MPS (Apple Silicon) or CUDA is available
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned MobileNetV3 model
num_classes = len(classes)  # Change this based on your number of classes
model = models.mobilenet_v3_large(pretrained=False)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model.load_state_dict(torch.load('../models/position_classifier_mobilenetv3.pth'))
model = model.to(device)
model.eval()

# Transform for inference (96x96 resizing and normalization)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((96, 96)),  # Resize to 96x96 pixels
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Function to run inference on a cv2 frame
def predict_frame(frame):

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = transform(image).unsqueeze(0).to(device)

    # Run the model in evaluation mode
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    # Get the predicted class
    predicted_class = classes[predicted.item()]
    return predicted_class

In [ ]:
import matplotlib.pyplot as plt
import cv2

image_path = '/Users/shingkai/code/personal-projects/mk8/race_videos/training/donut_plains/p1_position/0/002801.png'

frame = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
predicted_class = predict_frame(frame)
print(f"Predicted class: {predicted_class}")
